# **connection**

In [ ]:
pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 8.5 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 


In [ ]:
import json
from datetime import datetime

credentials = 'client_secret.json'

with open(credentials) as f:
    creds = json.load(f)

In [ ]:
import praw
reddit = praw.Reddit(client_id = creds['client_id'],
                    client_secret = creds['client_secret'],
                    user_agent = creds['user_agent'],
                    redirect_uri = creds['redirect_uri'],
                    refresh_token = creds['refresh_token'],
                     check_for_async = False)

# **Loading comments stream data**

In [ ]:
import pandas as pd

final_df = pd.read_csv('/content/com_stream_france.csv')

In [ ]:
# final_df = final_df[final_df.id.isna()==False]

In [ ]:
len(final_df)

In [ ]:
comment_ids = list(final_df.id)

In [ ]:
comment_fullnames = ["t1_"+i for i in comment_ids]

In [ ]:
# apr_df = final_df

In [ ]:
# apr_df = final_df[final_df['created_utc'] >= 1652161733] 

In [ ]:
# from datetime import datetime
# datetime.fromtimestamp(int(final_df.created_utc.min()))
# apr_df.created_utc.max()

# **Reddit API comment collection**

In [ ]:
def create_batch(total):
  final_batchs = []
  start = 0
  end = 500

  while len(total)-start > 500:
    batch = total[start:end]
    final_batchs.append(batch)
    start = end
    end = end+500

  last_batch = total[start:]
  final_batchs.append(last_batch)

  return final_batchs

In [ ]:
final_list = []
batches = create_batch(comment_fullnames)
print("Number of batches: ", len(batches))

b_count = 1
for batch in batches:
  com_generator = reddit.info(batch)

  for com in com_generator:
    com_dict={}
    com_dict['id'] = com.id
    com_dict['body'] = com.body

    final_list.append(com_dict)
  
  print("-----Batch: ", b_count, "----Num of comments collected: ", len(final_list))
  b_count+=1


In [ ]:
final_list = [{'id': i['id'], 'body': " ".join(i['body'].split())} for i in final_list]

In [ ]:
len(final_list)

129243

In [ ]:
reddit_collected_com = pd.DataFrame(final_list)

In [ ]:
reddit_collected_com.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129243 entries, 0 to 129242
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      129243 non-null  object
 1   body    129243 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


In [ ]:
# reddit_collected_com.to_csv('./collected_com_europe_before_10may_part6.csv', header=True, index=False, columns=list(reddit_collected_com.axes[1]))

In [ ]:
# from google.colab import files
# files.download("/content/collected_com_europe_before_10may_part6.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Merging files**

In [ ]:
# import pandas as pd

# temp1 = pd.read_csv("/content/collected_com_europe_before_10may_part1.csv")
# temp2 = pd.read_csv('/content/collected_com_europe_before_10may_part2.csv')
# temp3 = pd.read_csv("/content/collected_com_europe_before_10may_part3.csv")
# temp4 = pd.read_csv("/content/collected_com_europe_before_10may_part4.csv")
# temp5 = pd.read_csv("/content/collected_com_europe_before_10may_part5.csv")
# temp6 = pd.read_csv("/content/collected_com_europe_before_10may_part6.csv")

In [ ]:
# temp1 = temp1.append(temp2, ignore_index=True)

In [ ]:
# temp1 = temp1.append(temp3, ignore_index=True)

In [ ]:
# temp1 = temp1.append(temp4, ignore_index=True)

In [ ]:
# temp1 = temp1.append(temp5, ignore_index=True)

In [ ]:
# temp1 = temp1.append(temp6, ignore_index=True)

In [ ]:
# temp1.info()

In [ ]:
# temp1[temp1.body.isna()==True]

In [ ]:
# for i in reddit.info(['t1_hztvqeu', 't1_i3jgpir', 't1_i4lrlks']):
#   print(i.id, i.body)

In [ ]:
# temp1 = temp1[temp1.body.isna()==False]

In [ ]:
# temp1.to_csv('./collected_com_europe_before_10may_part_merged.csv', header=True, index=False, columns=list(temp1.axes[1]))

In [ ]:
# from google.colab import files
# files.download("/content/collected_com_europe_before_10may_part_merged.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Finding removed comments**

In [ ]:
removed_com = reddit_collected_com[reddit_collected_com['body']=='[removed]']
removed_id = removed_com['id']
print(len(removed_id))

3438


In [ ]:
removed_com

,id,body
8,i7ystov,[removed]
176,i7yuoos,[removed]
265,i7yvp39,[removed]
291,i7yvzgd,[removed]
379,i7yx8ba,[removed]
...,...,...
128882,i9tp2mj,[removed]
129001,i9tr61t,[removed]
129005,i9tr9cx,[removed]
129011,i9trcel,[removed]


In [ ]:
found_removed_com = final_df[final_df['id'].isin(set(list(removed_id)))]
print(len(found_removed_com))
found_removed_com.head()

3438


,author,body,collapsed,controversiality,created_utc,id,link_id,parent_id,permalink,score
8,20fxfljx,:D,0.0,0.0,1652129190.0,i7ystov,t3_uljfld,t1_i7yci3l,/r/europe/comments/uljfld/war_in_ukraine_megat...,1.0
176,20fxfljx,Strange. Because there was someone defending a...,0.0,0.0,1652129970.0,i7yuoos,t3_uljfld,t1_i7xuwun,/r/europe/comments/uljfld/war_in_ukraine_megat...,1.0
265,hxc0bdn2,Right but if all these insignificant countries...,0.0,0.0,1652130399.0,i7yvp39,t3_ulk92z,t1_i7ypdcy,/r/europe/comments/ulk92z/bulgaria_says_will_v...,1.0
291,5zqlkpcu,If you think I’m a “communist who defends Stal...,0.0,0.0,1652130522.0,i7yvzgd,t3_ulvc66,t1_i7yuyz8,/r/europe/comments/ulvc66/in_a_world_of_rivalr...,1.0
379,7slke0fq,> I'm Austrian so\n\nDid I ask bozo\n\nActuall...,0.0,0.0,1652131056.0,i7yx8ba,t3_ulwv9b,t1_i7ytyo8,/r/europe/comments/ulwv9b/13_countries_in_the_...,1.0


In [ ]:
found_removed_com.to_csv('./removed_com_europe_10to24may.csv', header=True, index=False, columns=list(found_removed_com.axes[1]))

In [ ]:
from google.colab import files
files.download("/content/removed_com_europe_10to24may.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Word count cells**

In [ ]:
# import numpy as np
# final_df['content'] = np.where(final_df['selftext'].isnull(), final_df['title'], final_df['title']+final_df['selftext'])

In [ ]:
from nltk.tokenize import WhitespaceTokenizer
     
tk = WhitespaceTokenizer()

# final_df['word_count'] = final_df.content.apply(lambda x: len(tk.tokenize(x)))
final_df['word_count'] = final_df.body.apply(lambda x: len(tk.tokenize(str(x))))

In [ ]:
print(final_df.word_count.sum())

4184462


# **Pushshift API calls method for collection using comment ID**

In [ ]:
#GIVING WRONG DATA

# import requests

# batches = create_batch(comment_ids)

# for batch in batches:
#   # parameters = {"ids" : batch, "fields" : ['body', 'author','created_utc', 'id','permalink','score']}
#   response = requests.get("https://ssl.reddit.com/r/france/api/info.json", params=parameters)
#   print("Request status: ", response.status_code)
#   res = response.json()
#   print("----Num of comments fetched: ", len(res['data']))

# **Reddit.comment() function to collect using comment ID**

In [ ]:
def find_comment(id):
  comment = reddit.comment(str(id))
  return comment

In [ ]:
reddit_com_list = []
count = 0
for i in comment_ids[0:9611]:
  com = find_comment(i)

  com_dict={}
  com_dict['id'] = com.id
  com_dict['parent_id'] = com.parent_id
  com_dict['link_id'] = com.link_id
  com_dict['body'] = com.body
  com_dict['collapsed'] = 0 if com.collapsed == False else 1
  com_dict['score'] = com.score
  com_dict['controversiality'] = com.controversiality
  com_dict['permalink'] = com.permalink
  com_dict['created_utc'] = com.created_utc
  try: 
    if com.author != None:
      com_dict['author'] = com.author.id
    else:
      com_dict['author'] = 'Not found'

  except Exception as e:
    print("Author ERROR: ", e)
    print("Comment number:" + com.id)
    com_dict['author'] = 'Not found'

  reddit_com_list.append(com_dict)
  count+=1
  print("********COUNT: ", count)

In [ ]:
reddit_com_df = pd.DataFrame(reddit_com_list)

In [ ]:
reddit_com_df

In [ ]:
reddit_com_df.to_csv('./1reddit_com_france_9to15may.csv', header=True, index=False, columns=list(reddit_com_df.axes[1]))

In [ ]:
from google.colab import files
files.download("/content/1reddit_com_france_9to15may.csv")

In [ ]:
removed_com = reddit_com_df[reddit_com_df['body']=='[removed]']
removed_id = removed_com['id']
print(len(removed_id))

In [ ]:
found_removed_com = apr_df[apr_df['id'].isin(set(list(removed_id)))]
print(len(found_removed_com))
found_removed_com.head()

In [ ]:
found_removed_com.to_csv('./removed_com_france_9to15may.csv', header=True, index=False, columns=list(found_removed_com.axes[1]))

In [ ]:
from google.colab import files
files.download("/content/removed_com_france_9to15may.csv")

# **Old Pushshift (pmaw) method for collection using comment ID**

In [ ]:
# comment_ids = [id for id in comment_ids if type(id) == str]

In [ ]:
# final_df = final_df.sort_values(['created_utc'], ascending=True)
# final_df.created_utc.max()

In [ ]:
pip install pmaw==2.1.2

In [ ]:
import pandas as pd
from pmaw import PushshiftAPI

pushshift_api = PushshiftAPI()
comment = api.search_comments(ids=comment_ids[0])

In [ ]:
import datetime as dt

# after = int(dt.datetime(2022,5,9,0,0,0).timestamp())
after = int(1652129200.0)
before = int(1652671820.0)
subreddit = 'france'

In [ ]:
comments = pushshift_api.search_comments(subreddit=subreddit, after=after, before = before)

In [ ]:
comments_df = pd.DataFrame(comments)

comments_df_filtered = comments_df[["id","author","body", "collapsed", 'score',"controversiality", "parent_id", "link_id","permalink","created_utc"]]

# comments_df_filtered.created_utc = comments_df_filtered.created_utc.apply(lambda x: dt.datetime.fromtimestamp(x))
comments_df_filtered.head()

In [ ]:
len(comments_df_filtered)

39104

In [ ]:
found_comments = comments_df_filtered[comments_df_filtered['id'].isin(set(comment_ids))]

In [ ]:
len(found_comments)

38072

In [ ]:
removed = found_comments[found_comments['body']=='[removed]']
removed_id = removed['id']
print(len(removed_id))

82


In [ ]:
removed_com_stream = final_df[final_df['id'].isin(list(removed_id))]
print(len(removed_com_stream))
removed_com_stream.head()

82


,author,body,collapsed,controversiality,created_utc,id,link_id,parent_id,permalink,score
547,dr1m7ze2,C'est quand même dingue qu'on puisse reprocher...,0.0,0.0,1.652142e+09,i7zll14,t3_ulw3zl,t3_ulw3zl,/r/france/comments/ulw3zl/qui_est_le_trou_des_...,1.0
548,dr1m7ze2,Je ne vois pas trop ce que tu reproches à la n...,0.0,0.0,1.652142e+09,i7zlwm9,t3_ulw3zl,t1_i7yd5dt,/r/france/comments/ulw3zl/qui_est_le_trou_des_...,1.0
549,dr1m7ze2,"Ce n'est pas mon amie, et je pense être totale...",0.0,0.0,1.652142e+09,i7zmo3p,t3_ulw3zl,t1_i7yes9a,/r/france/comments/ulw3zl/qui_est_le_trou_des_...,1.0
1290,h40nuvbg,"Y a d'autre solution que les capotes, le ring,...",0.0,0.0,1.652167e+09,i80v1b4,t3_ulw3zl,t3_ulw3zl,/r/france/comments/ulw3zl/qui_est_le_trou_des_...,1.0
3659,9blu8xis,"D'accord, donc non seulement tu ne comprends r...",0.0,0.0,1.652183e+09,i81ewlm,t3_ulkxbk,t1_i8161es,/r/france/comments/ulkxbk/législatives_fabien_...,1.0


In [ ]:
removed.to_csv("./temp_france.csv", header = True, index=False, columns=list(removed.axes[1]))

In [ ]:
removed_com_stream.to_csv('./removed_com_france_9to15may.csv', header=True, index=False, columns=list(removed_com_stream.axes[1]))

# Deleted comments

In [ ]:
# deleted = found_comments[found_comments['body']=='[deleted]']
# deleted_id = deleted['id']
# print(len(deleted_id))

0


In [ ]:
# deleted_com_stream = final_df[final_df['id'].isin(list(deleted_id))]
# print(len(deleted_com_stream))
# deleted_com_stream.head()

0


,body,collapsed,controversiality,created_utc,id,link_id,parent_id,permalink,score


In [ ]:
# deleted_com_stream.to_csv('./deleted_com_science_29to5apr.csv', header=True, index=False, columns=list(deleted_com_stream.axes[1]))

# **Fetching parent id**

In [ ]:
parent_com_id = [id[3:] for id in removed_com_stream['parent_id'] if id[:2] == 't1']

NameError: ignored

In [ ]:
parent_com_arr = pushshift_api.search_comments(ids=parent_com_id)